In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 91% 135M/149M [00:01<00:00, 102MB/s]
100% 149M/149M [00:01<00:00, 110MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/brain-tumor-mri-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten,BatchNormalization,Dropout,Conv2D,MaxPooling2D
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import ConvNeXtLarge
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/Training',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/content/Testing',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
base_model = Xception(input_shape=(299,299,3), weights='imagenet', include_top=False)

In [ ]:
base_model.trainable = True

# Flag to control when to start fine-tuning
set_trainable = False

# Unfreeze layers from a specific layer onwards
for layer in base_model.layers:
    if layer.name == 'block11_sepconv1':  # Replace with the layer name from where you want to start fine-tuning
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Print the trainable status of each layer
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_layer_4 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_16 False
block2_pool False
batch_normalization_16 False
add_48 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_17 False
block3_pool False
batch_normalization_17 False
add_49 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_18 False
block4_pool False
batch_normalization_18 False
add_50 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn

In [ ]:
model = Sequential()

model.add(base_model)
model.add(Flatten())
model.add(Dense(124,activation='relu'))
model.add(Dense(124,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 149s 802ms/step - accuracy: 0.9892 - loss: 0.0430 - val_accuracy: 0.9779 - val_loss: 0.1595
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 147s 800ms/step - accuracy: 0.9793 - loss: 0.0910 - val_accuracy: 0.9619 - val_loss: 0.1490
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 204s 809ms/step - accuracy: 0.9898 - loss: 0.0348 - val_accuracy: 0.9931 - val_loss: 0.0366
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 202s 803ms/step - accuracy: 0.9918 - loss: 0.0258 - val_accuracy: 0.9771 - val_loss: 0.1681
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 202s 806ms/step - accuracy: 0.9926 - loss: 0.0315 - val_accuracy: 0.9794 - val_loss: 0.0756
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 148s 802ms/step - accuracy: 0.9923 - loss: 0.0333 - val_accuracy: 0.9886 - val_loss: 0.0609
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 147s 797ms/step - accuracy: 0.9928 - loss: 0.0216 - val_accuracy: 0.9855 - val_loss: 0.0492
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 147s 800ms/step - accuracy: 0.9923 -

In [39]:

model.save('brain_tumor_final2.h5')


In [ ]:


from google.colab import files
files.download('brain_tumor_final2.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>